In [1]:
import os
import random
import pickle
import torch
import torchvision
import pandas as pd
import numpy as np

In [2]:
with open('../data/pickles/main_dict.pickle', 'rb') as f:
    main_dict = pickle.load(f)

In [3]:
def get_score(x, y):
    answer_tags = main_dict[x]['all_tags']
    query_tags = main_dict[y]['all_tags']
    score = 0
    i = 0
    for tag in answer_tags:
        if tag in query_tags:
            i += 1
    if i == 0:
        return 0

    precision = i / len(answer_tags)
    recall = i / len(query_tags)
    score = 2 * precision * recall / (precision + recall)
    return score
        

In [4]:
get_score(190, 621)

0.7407407407407407

In [5]:
positiv_pairs = dict()
main_list = list(main_dict.keys())
for i in range(len(main_dict)):
    idx_1 = main_list[i]
    if len(main_dict[idx_1]['all_tags']) < 3:
        continue
    positiv_pairs[idx_1] = list()
    for j in range(len(main_dict)):
        idx_2 = main_list[j]
        if idx_2 == idx_1:
            continue
        if set(main_dict[idx_1]['all_genres']) != set(main_dict[idx_2]['all_genres']):
            continue
        score = get_score(idx_1, idx_2)
        if score >= 0.4:
            positiv_pairs[idx_1].append(idx_2)
    if len(positiv_pairs[idx_1]) == 0:
        positiv_pairs.pop(idx_1)
            
            
        
            

KeyboardInterrupt: 

In [ ]:
len(positiv_pairs)

In [96]:
with open('../pickles_for_git/positiv.p', 'wb') as f:
    pickle.dump(positiv_pairs, f)

In [97]:
negativ_pairs = dict()
main_list = list(main_dict.keys())
for i in range(len(main_dict)):
    idx_1 = main_list[i]
    if len(main_dict[idx_1]['all_tags']) < 3:
        continue
    negativ_pairs[idx_1] = list()
    for j in range(len(main_dict)):
        idx_2 = main_list[j]
        if idx_2 == idx_1:
            continue
        if len(set(main_dict[idx_1]['all_tags']) & set(main_dict[idx_2]['all_tags'])) == 0:
            negativ_pairs[idx_1].append(idx_2)
    if len(negativ_pairs[idx_1]) == 0:
        negativ_pairs.pop(idx_1)

In [98]:
len(negativ_pairs)

6843

In [99]:
with open('../pickles_for_git/negativ.p', 'wb') as f:
    pickle.dump(negativ_pairs, f)

In [6]:
df_train = pd.read_csv('../data/csv/train.csv')
df_train.shape

(7897, 2049)

In [7]:
df_train.sample(5)

,id,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
3296,26008,4.617371,8.720132,8.669876,0.992930,0.000000,2.674375,7.227406,2.502406,0.901238,...,3.745683,5.172704,6.297438,5.225374,0.000000,0.000000,0.0,1.590548,0.000000,0.189794
1458,80000,3.929991,2.650877,6.394147,2.109811,0.000000,1.332203,2.111559,3.409339,0.119490,...,2.503194,5.912471,7.428972,1.818782,1.278589,3.234374,0.0,1.734027,0.653902,1.996290
1517,81555,3.082262,1.950925,7.217719,0.271827,0.082024,0.000000,5.266066,0.000000,2.872050,...,3.409176,0.631545,10.590041,6.902850,0.807486,1.786445,0.0,1.934831,2.492859,0.834827
3897,125160,3.129672,6.073639,5.953879,2.659724,2.994831,0.195992,2.974214,1.041844,2.424673,...,4.002719,4.819555,4.598737,2.688082,3.801705,0.000000,0.0,5.789520,2.482530,1.525012
6270,132456,0.000000,2.319759,4.775220,1.698771,1.386433,0.000000,1.952732,0.000000,0.000000,...,0.000000,5.881092,5.072213,1.153139,6.719053,7.849669,0.0,4.607787,2.203102,5.885794


In [8]:
train_idx = df_train.id.values

In [9]:
train_idx

array([     5,     10,    140, ..., 153337, 154303, 154306])

In [10]:
np.random.choice(train_idx, 1, replace=False)[0]

133976

In [12]:
with open('../data/pickles/positiv.p', 'rb') as f:
    positiv_pairs = pickle.load(f)

with open('../data/pickles/negativ.p', 'rb') as f:
    negativ_pairs = pickle.load(f)
    
with open('../data/pickles/triplets.p', 'rb') as f:
    triplets = pickle.load(f)

with open('../data/pickles/val_triplets.p', 'rb') as f:
    val_triplets = pickle.load(f)

In [13]:
all_trpl = triplets + val_triplets

In [14]:
len(all_trpl)

12000

In [15]:
i = 0
triplets_2 = list()
while i < 10000:
    query_id = np.random.choice(train_idx, 1, replace=False)[0]
    try:
        p_id = np.random.choice(positiv_pairs[query_id], 1, replace=False)[0]
        n_id = np.random.choice(negativ_pairs[query_id], 1, replace=False)[0]
    except KeyError:
        continue
    
    if p_id not in train_idx or n_id not in train_idx:
        continue
    if (query_id, p_id, n_id) not in all_trpl and (query_id, p_id, n_id) not in triplets_2:
        triplets_2.append((query_id, p_id, n_id))
        i += 1

In [16]:
len(triplets_2)

10000

In [17]:
len(set([x[0] for x in triplets_2]))

5092

In [18]:
triplets_2[0]

(69789, 30095, 141616)

In [22]:
def find_img_path(idx):
    name = f'{idx:06}.png'
    base_path = '../data/spectrograms/train/'
    for address, dirs, files in os.walk(base_path):
        for img in files:
            if img == name:
                img_path = address + '/' + img
                return img_path
    print(name)

In [27]:
trpl_with_path = [(find_img_path(x[0]), find_img_path(x[1]), find_img_path(x[2])) for x in triplets_2]

In [28]:
trpl_with_path[0]

('../data/spectrograms/train/train/Experimental/069789.png',
 '../data/spectrograms/train/train/Experimental/030095.png',
 '../data/spectrograms/train/train/Electronic/141616.png')

In [29]:
triplets_big = triplets + trpl_with_path

In [31]:
with open('../data/pickles/triplets_big.p', 'wb') as f:
    pickle.dump(triplets_big, f)

In [153]:
with open('../pickles_for_git/triplets.p', 'rb') as f:
    tripls = pickle.load(f)

In [154]:
tripls[0]

('../data/spectrograms/train/train/Instrumental/141139.png',
 '../data/spectrograms/train/train/Instrumental/111793.png',
 '../data/spectrograms/train/train/Pop/046842.png')

In [3]:
with open('../data/pickles/val_triplets.p', 'rb') as f:
    val_triplets = pickle.load(f)
    
with open('../data/pickles/triplets_big.p', 'rb') as f:
    triplets = pickle.load(f)

In [8]:
for i in triplets:
    print([x[3:] for x in i])
    break

['data/spectrograms/train/train/Instrumental/141139.png', 'data/spectrograms/train/train/Instrumental/111793.png', 'data/spectrograms/train/train/Pop/046842.png']


In [7]:
triplets[0]

('../data/spectrograms/train/train/Instrumental/141139.png',
 '../data/spectrograms/train/train/Instrumental/111793.png',
 '../data/spectrograms/train/train/Pop/046842.png')